In [1]:
import pandas as pd
import jdatetime
import matplotlib.pyplot as plt
import os
import matplotlib.dates as mdates
import numpy as np

# Load the Excel file
excel_path = "pizometer_data.xlsx"
sheets = pd.read_excel(excel_path, sheet_name=None)

# Create folder for plots
os.makedirs("plots", exist_ok=True)

# Convert Shamsi (Jalali) date to Gregorian
def shamsi_to_gregorian_safe(date_str):
    try:
        year, month, day = map(int, str(date_str).split('/'))
        return jdatetime.date(year, month, day).togregorian()
    except:
        return None

# Process each sheet
for sheet_name, df in sheets.items():
    if 'tarikh' not in df.columns or 'taraz' not in df.columns:
        print(f"⛔ Skipping '{sheet_name}': Missing 'tarikh' or 'taraz'")
        continue

    df['tarikh_miladi'] = df['tarikh'].apply(shamsi_to_gregorian_safe)
    df = df.dropna(subset=['tarikh_miladi', 'taraz']).copy()

    # Make sure 'taraz' is numeric
    df['taraz'] = pd.to_numeric(df['taraz'], errors='coerce')
    df = df.dropna(subset=['taraz'])

    if len(df) < 2:
        print(f"⚠️ Not enough data in '{sheet_name}' for plotting.")
        continue

    df = df.sort_values('tarikh_miladi')

    x = mdates.date2num(df['tarikh_miladi'])
    y = df['taraz'].values

    # Plot
    plt.figure(figsize=(14, 4))
    plt.scatter(df['tarikh_miladi'], df['taraz'], color='blue', label='Observations')

    # Fit regression line if data is not constant
    if len(x) >= 2 and np.std(y) > 1e-6:
        try:
            coeffs = np.polyfit(x, y, 1)
            trendline = np.poly1d(coeffs)
            y_trend = trendline(x)
            plt.plot(df['tarikh_miladi'], y_trend, color='red', linestyle='--', label='Trendline')
        except np.linalg.LinAlgError:
            print(f"⚠️ Trendline failed for '{sheet_name}' (SVD issue)")

    plt.xlabel('Gregorian Date')
    plt.ylabel('Taraz (m)')
    plt.title("Piezometer Observation")
    plt.legend()
    plt.grid(True)

    # Save plot
    plt.savefig(f"plots/{sheet_name}.png", bbox_inches='tight', dpi=300)
    plt.close()

    # Save cleaned DataFrame back into dictionary if needed later
    sheets[sheet_name] = df

print("✅ All plots saved successfully in 'plots/' folder.")


✅ All plots saved successfully in 'plots/' folder.


In [3]:
import pandas as pd
import jdatetime
import matplotlib.pyplot as plt
import os
import matplotlib.dates as mdates
import numpy as np

# Load only the target sheet
excel_path = "pizometer_data.xlsx"
target_sheet = "جمع آب"
df = pd.read_excel(excel_path, sheet_name=target_sheet)

# Create folder for plots
os.makedirs("plots", exist_ok=True)

# Convert Shamsi (Jalali) date to Gregorian
def shamsi_to_gregorian_safe(date_str):
    try:
        year, month, day = map(int, str(date_str).split('/'))
        return jdatetime.date(year, month, day).togregorian()
    except:
        return None

# Validate required columns
if 'tarikh' not in df.columns or 'taraz' not in df.columns:
    print(f"⛔ Skipping: Missing 'tarikh' or 'taraz' in sheet '{target_sheet}'")
else:
    df['tarikh_miladi'] = df['tarikh'].apply(shamsi_to_gregorian_safe)
    df = df.dropna(subset=['tarikh_miladi', 'taraz']).copy()

    df['taraz'] = pd.to_numeric(df['taraz'], errors='coerce')
    df = df.dropna(subset=['taraz'])

    if len(df) < 2:
        print(f"⚠️ Not enough data in '{target_sheet}' for plotting.")
    else:
        df = df.sort_values('tarikh_miladi')
        x = mdates.date2num(df['tarikh_miladi'])
        y = df['taraz'].values

        # Plot
        plt.figure(figsize=(14, 4))
        plt.scatter(df['tarikh_miladi'], df['taraz'], color='blue', label='Observations')

        # Add trendline if data is valid
        if np.std(y) > 1e-6:
            try:
                coeffs = np.polyfit(x, y, 1)
                slope_per_day = coeffs[0]
                slope_per_year = slope_per_day * 365.25
                trendline = np.poly1d(coeffs)
                y_trend = trendline(x)
                plt.plot(df['tarikh_miladi'], y_trend, color='red', linestyle='--',
                         label=f'Trendline (annual rate: {slope_per_year:.2f} m/year)')
            except np.linalg.LinAlgError:
                print(f"⚠️ Trendline failed for '{target_sheet}' (SVD issue)")

        plt.xlabel('Gregorian Date')
        plt.ylabel('Taraz (m)')
        plt.title(f"Piezometer Observation – {target_sheet}")
        plt.legend()
        plt.grid(True)

        # Save plot
        plt.savefig(f"plots/{target_sheet}.png", bbox_inches='tight', dpi=300)
        plt.close()

        print(f"✅ Plot for '{target_sheet}' saved in 'plots/' folder.")


✅ Plot for 'جمع آب' saved in 'plots/' folder.


In [8]:
# ... (previous code remains unchanged until the plotting section)

# Plot
plt.figure(figsize=(14, 4))
plt.scatter(df['tarikh_miladi'], df['taraz'], color='blue', label='Observations')

# Process each period
for label, start_date, end_date in periods:
    # Filter data for the period
    mask = pd.Series(True, index=df.index)
    if start_date:
        mask &= df['tarikh_miladi'] >= pd.to_datetime(start_date)
    if end_date:
        mask &= df['tarikh_miladi'] <= pd.to_datetime(end_date)
    
    df_period = df[mask].copy()
    
    if len(df_period) >= 2:
        x = mdates.date2num(df_period['tarikh_miladi'])
        y = df_period['taraz'].values

        # Calculate trendline if data is valid
        if np.std(y) > 1e-6:
            try:
                coeffs = np.polyfit(x, y, 1)
                slope_per_day = coeffs[0]
                slope_per_year = slope_per_day * 365.25
                trendline = np.poly1d(coeffs)
                y_trend = trendline(x)
                plt.plot(df_period['tarikh_miladi'], y_trend, linestyle='--',
                        linewidth=2.5,  # Increased trendline width
                        label=f'{label} (annual rate: {slope_per_year:.2f} m/year)')
            except np.linalg.LinAlgError:
                print(f"⚠️ Trendline failed for '{label}' in '{target_sheet}' (SVD issue)")
        else:
            print(f"⚠️ Not enough variation in '{label}' for trendline.")
    else:
        print(f"⚠️ Not enough data in '{label}' for trendline.")

plt.xlabel('Gregorian Date')
plt.ylabel('Taraz (m)')
plt.title(f"Piezometer Observation – {target_sheet}")
plt.legend()
plt.grid(True)

# Save plot
plt.savefig(f"plots/{target_sheet}_segmented_trend.png", bbox_inches='tight', dpi=300)
plt.close()

print(f"✅ Plot with segmented trendlines for '{target_sheet}' saved in 'plots/' folder.")

✅ Plot with segmented trendlines for 'جمع آب' saved in 'plots/' folder.


In [6]:
import pandas as pd
import jdatetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import os

# Load only the target sheet
excel_path = "pizometer_data.xlsx"
target_sheet = "جمع آب"
df = pd.read_excel(excel_path, sheet_name=target_sheet)

# Create folder for plots
os.makedirs("plots", exist_ok=True)

# Convert Shamsi (Jalali) date to Gregorian
def shamsi_to_gregorian_safe(date_str):
    try:
        year, month, day = map(int, str(date_str).split('/'))
        return jdatetime.date(year, month, day).togregorian()
    except:
        return None

# Validate required columns
if 'tarikh' not in df.columns or 'taraz' not in df.columns:
    print(f"⛔ Skipping: Missing 'tarikh' or 'taraz' in sheet '{target_sheet}'")
else:
    df['tarikh_miladi'] = df['tarikh'].apply(shamsi_to_gregorian_safe)
    df = df.dropna(subset=['tarikh_miladi', 'taraz']).copy()

    # Convert tarikh_miladi to pandas Timestamp for consistent comparison
    df['tarikh_miladi'] = pd.to_datetime(df['tarikh_miladi'])
    
    df['taraz'] = pd.to_numeric(df['taraz'], errors='coerce')
    df = df.dropna(subset=['taraz'])

    if len(df) < 2:
        print(f"⚠️ Not enough data in '{target_sheet}' for plotting.")
    else:
        df = df.sort_values('tarikh_miladi')

        # Define periods for trendlines
        periods = [
            ('1965 to 1975', None, '1975-12-31'),
            ('1982 to 1997', '1982-01-01', '1997-12-31'),
            ('2003 to 2022', '2003-01-01', None)
        ]

        # Define color dictionary for trendlines
        trendline_colors = {
            '1965 to 1975': 'red',
            '1982 to 1997': 'green',
            '2003 to 2022': 'purple'
        }

        # Plot
        plt.figure(figsize=(14, 4))
        plt.scatter(df['tarikh_miladi'], df['taraz'], color='blue', label='Observations')

        # Process each period
        for label, start_date, end_date in periods:
            # Filter data for the period
            mask = pd.Series(True, index=df.index)
            if start_date:
                mask &= df['tarikh_miladi'] >= pd.to_datetime(start_date)
            if end_date:
                mask &= df['tarikh_miladi'] <= pd.to_datetime(end_date)
            
            df_period = df[mask].copy()
            
            if len(df_period) >= 2:
                x = mdates.date2num(df_period['tarikh_miladi'])
                y = df_period['taraz'].values

                # Get start and end years for the period
                start_year = df_period['tarikh_miladi'].min().year
                end_year = df_period['tarikh_miladi'].max().year

                # Calculate trendline if data is valid
                if np.std(y) > 1e-6:
                    try:
                        coeffs = np.polyfit(x, y, 1)
                        slope_per_day = coeffs[0]
                        slope_per_year = slope_per_day * 365.25
                        trendline = np.poly1d(coeffs)
                        y_trend = trendline(x)
                        plt.plot(df_period['tarikh_miladi'], y_trend, linestyle='--',
                                linewidth=2.5,
                                color=trendline_colors[label],
                                label=f'{label} ({start_year}-{end_year}, rate: {slope_per_year:.2f} m/year)')
                    except np.linalg.LinAlgError:
                        print(f"⚠️ Trendline failed for '{label}' in '{target_sheet}' (SVD issue)")
                else:
                    print(f"⚠️ Not enough variation in '{label}' for trendline.")
            else:
                print(f"⚠️ Not enough data in '{label}' for trendline.")

        plt.xlabel('Gregorian Date')
        plt.ylabel('Taraz (m)')
        plt.title(f"Piezometer Observation – {target_sheet}")
        plt.legend()
        plt.grid(True)

        # Save plot
        plt.savefig(f"plots/{target_sheet}_segmented_trend.png", bbox_inches='tight', dpi=300)
        plt.close()

        print(f"✅ Plot with segmented trendlines for '{target_sheet}' saved in 'plots/' folder.")

✅ Plot with segmented trendlines for 'جمع آب' saved in 'plots/' folder.


In [15]:
start_year = df['tarikh_miladi'].min().year
end_year = df['tarikh_miladi'].max().year

In [16]:
print(end_year)

2022


In [17]:
ذهاب 2

,سال,ماه,روز,سال_آ,کد_مح,محدود,نام_م,سطح_آ,XUTM,YUTM,علت_ع,تراز__,ملاحظ,تيسن__,تيسن_1,taraz,tarikh,date,subsidence,tarikh_miladi
3,1344,7,9,1344-45,6007,مشهد - چناران,جمع آب,73.64,690870,4054367,0,1203.73,0,35.19,2222.1,1130.09,1344/7/9,2015-04-21,-3.32,1965-10-01
2,1344,8,10,1344-45,6007,مشهد - چناران,جمع آب,73.70,690870,4054367,0,1203.73,0,35.19,2222.1,1130.03,1344/8/10,2015-03-28,-1.49,1965-11-01
1,1344,9,9,1344-45,6007,مشهد - چناران,جمع آب,73.68,690870,4054367,0,1203.73,0,35.19,2222.1,1130.05,1344/9/9,2015-03-04,-0.65,1965-11-30
5,1344,10,6,1344-45,6007,مشهد - چناران,جمع آب,73.62,690870,4054367,0,1203.73,0,35.19,2222.1,1130.11,1344/10/6,2015-07-02,6.80,1965-12-27
4,1344,11,7,1344-45,6007,مشهد - چناران,جمع آب,73.56,690870,4054367,0,1203.73,0,35.19,2222.1,1130.17,1344/11/7,2015-05-15,-1.89,1966-01-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,1401,2,15,1400-1,6007,مشهد - چناران,جمع آب,96.34,690870,4054367,0,1203.73,0,35.19,2222.1,1107.39,1401/2/15,NaT,NaN,2022-05-05
500,1401,3,17,1400-1,6007,مشهد - چناران,جمع آب,96.60,690870,4054367,0,1203.73,0,35.19,2222.1,1107.13,1401/3/17,NaT,NaN,2022-06-07
501,1401,4,15,1400-1,6007,مشهد - چناران,جمع آب,96.91,690870,4054367,0,1203.73,0,35.19,2222.1,1106.82,1401/4/15,NaT,NaN,2022-07-06
504,1401,5,11,1400-1,6007,مشهد - چناران,جمع آب,97.40,690870,4054367,0,1203.73,0,35.19,2222.1,1106.33,1401/5/11,NaT,NaN,2022-08-02


In [22]:
import pandas as pd
import jdatetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import os

# Load only the target sheet
excel_path = "pizometer_data.xlsx"
target_sheet = "ذهاب 2"
df = pd.read_excel(excel_path, sheet_name=target_sheet)

# Create folder for plots
os.makedirs("plots", exist_ok=True)

# Convert Shamsi (Jalali) date to Gregorian
def shamsi_to_gregorian_safe(date_str):
    try:
        year, month, day = map(int, str(date_str).split('/'))
        return jdatetime.date(year, month, day).togregorian()
    except:
        return None

# Validate required columns
if 'tarikh' not in df.columns or 'taraz' not in df.columns:
    print(f"⛔ Skipping: Missing 'tarikh' or 'taraz' in sheet '{target_sheet}'")
else:
    df['tarikh_miladi'] = df['tarikh'].apply(shamsi_to_gregorian_safe)
    df = df.dropna(subset=['tarikh_miladi', 'taraz']).copy()

    # Convert tarikh_miladi to pandas Timestamp for consistent comparison
    df['tarikh_miladi'] = pd.to_datetime(df['tarikh_miladi'])
    
    df['taraz'] = pd.to_numeric(df['taraz'], errors='coerce')
    df = df.dropna(subset=['taraz'])

    if len(df) < 2:
        print(f"⚠️ Not enough data in '{target_sheet}' for plotting.")
    else:
        df = df.sort_values('tarikh_miladi')

        # Define periods for trendlines
        periods = [
            ('1965 to 1975', None, '1975-12-31'),
            ('1982 to 1997', '1982-01-01', '1997-12-31'),
            ('2003 to 2022', '2003-01-01', None)
        ]

        # Define color dictionary for trendlines
        trendline_colors = {
            '1965 to 1975': 'red',
            '1982 to 1997': 'green',
            '2003 to 2022': 'purple'
        }

        # Plot
        plt.figure(figsize=(14, 4))
        plt.scatter(df['tarikh_miladi'], df['taraz'], color='blue', label='Observations')

        # Process each period
        for label, start_date, end_date in periods:
            # Filter data for the period
            mask = pd.Series(True, index=df.index)
            if start_date:
                mask &= df['tarikh_miladi'] >= pd.to_datetime(start_date)
            if end_date:
                mask &= df['tarikh_miladi'] <= pd.to_datetime(end_date)
            
            df_period = df[mask].copy()
            
            if len(df_period) >= 2:
                x = mdates.date2num(df_period['tarikh_miladi'])
                y = df_period['taraz'].values

                # Get start and end years for the period
                start_year = df_period['tarikh_miladi'].min().year
                end_year = df_period['tarikh_miladi'].max().year

                # Calculate trendline if data is valid
                if np.std(y) > 1e-6:
                    try:
                        coeffs = np.polyfit(x, y, 1)
                        slope_per_day = coeffs[0]
                        slope_per_year = slope_per_day * 365.25
                        trendline = np.poly1d(coeffs)
                        y_trend = trendline(x)
                        plt.plot(df_period['tarikh_miladi'], y_trend, linestyle='--',
                                linewidth=2.5,
                                color=trendline_colors[label],
                                label=f'{label} ({start_year}-{end_year}, rate: {slope_per_year:.2f} m/year)')
                    except np.linalg.LinAlgError:
                        print(f"⚠️ Trendline failed for '{label}' in '{target_sheet}' (SVD issue)")
                else:
                    print(f"⚠️ Not enough variation in '{label}' for trendline.")
            else:
                print(f"⚠️ Not enough data in '{label}' for trendline.")

        plt.xlabel('Gregorian Date')
        plt.ylabel('Taraz (m)')
        plt.title(f"Piezometer Observation – {target_sheet}")
        plt.legend()
        plt.grid(True)

        # Save plot
        plt.savefig(f"plots/{target_sheet}_segmented_trend.png", bbox_inches='tight', dpi=300)
        plt.close()

        print(f"✅ Plot with segmented trendlines for '{target_sheet}' saved in 'plots/' folder.")

⚠️ Not enough data in '1965 to 1975' for trendline.
⚠️ Not enough data in '1982 to 1997' for trendline.
✅ Plot with segmented trendlines for 'ذهاب 2' saved in 'plots/' folder.


In [23]:
import pandas as pd
import jdatetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import os

# Load only the target sheet
excel_path = "pizometer_data.xlsx"
target_sheet = "ذهاب 2"
df = pd.read_excel(excel_path, sheet_name=target_sheet)

# Create folder for plots
os.makedirs("plots", exist_ok=True)

# Convert Shamsi (Jalali) date to Gregorian
def shamsi_to_gregorian_safe(date_str):
    try:
        year, month, day = map(int, str(date_str).split('/'))
        return jdatetime.date(year, month, day).togregorian()
    except:
        return None

# Validate required columns
if 'tarikh' not in df.columns or 'taraz' not in df.columns:
    print(f"⛔ Skipping: Missing 'tarikh' or 'taraz' in sheet '{target_sheet}'")
else:
    df['tarikh_miladi'] = df['tarikh'].apply(shamsi_to_gregorian_safe)
    df = df.dropna(subset=['tarikh_miladi', 'taraz']).copy()

    # Convert tarikh_miladi to pandas Timestamp for consistent comparison
    df['tarikh_miladi'] = pd.to_datetime(df['tarikh_miladi'])
    
    df['taraz'] = pd.to_numeric(df['taraz'], errors='coerce')
    df = df.dropna(subset=['taraz'])

    if len(df) < 2:
        print(f"⚠️ Not enough data in '{target_sheet}' for plotting.")
    else:
        df = df.sort_values('tarikh_miladi')

        # Define periods for trendlines
        periods = [
            ('Main Data (Excluding 2017 to 2020-05)', None, None),  # Full range, will filter out noise
            ('Noise Data (2017 to 2020-05)', '2017-01-01', '2020-05-31')
        ]

        # Define color dictionary for trendlines
        trendline_colors = {
            'Main Data (Excluding 2017 to 2020-05)': 'red',
            'Noise Data (2017 to 2020-05)': 'green'
        }

        # Plot
        plt.figure(figsize=(14, 4))
        plt.scatter(df['tarikh_miladi'], df['taraz'], color='blue', label='Observations')

        # Process each period
        for label, start_date, end_date in periods:
            # Filter data for the period
            mask = pd.Series(True, index=df.index)
            if label == 'Main Data (Excluding 2017 to 2020-05)':
                # Exclude noisy data between 2017 and 2020-05
                mask &= ~((df['tarikh_miladi'] >= pd.to_datetime('2017-01-01')) & 
                          (df['tarikh_miladi'] <= pd.to_datetime('2020-05-31')))
            else:
                # Include only noisy data for 2017 to 2020-05
                if start_date:
                    mask &= df['tarikh_miladi'] >= pd.to_datetime(start_date)
                if end_date:
                    mask &= df['tarikh_miladi'] <= pd.to_datetime(end_date)
            
            df_period = df[mask].copy()
            
            if len(df_period) >= 2:
                x = mdates.date2num(df_period['tarikh_miladi'])
                y = df_period['taraz'].values

                # Get start and end years for the period
                start_year = df_period['tarikh_miladi'].min().year
                end_year = df_period['tarikh_miladi'].max().year

                # Calculate trendline if data is valid
                if np.std(y) > 1e-6:
                    try:
                        coeffs = np.polyfit(x, y, 1)
                        slope_per_day = coeffs[0]
                        slope_per_year = slope_per_day * 365.25
                        trendline = np.poly1d(coeffs)
                        y_trend = trendline(x)
                        plt.plot(df_period['tarikh_miladi'], y_trend, linestyle='--',
                                linewidth=2.5,
                                color=trendline_colors[label],
                                label=f'{label} ({start_year}-{end_year}, rate: {slope_per_year:.2f} m/year)')
                    except np.linalg.LinAlgError:
                        print(f"⚠️ Trendline failed for '{label}' in '{target_sheet}' (SVD issue)")
                else:
                    print(f"⚠️ Not enough variation in '{label}' for trendline.")
            else:
                print(f"⚠️ Not enough data in '{label}' for trendline.")

        plt.xlabel('Gregorian Date')
        plt.ylabel('Taraz (m)')
        plt.title(f"Piezometer Observation – {target_sheet}")
        plt.legend()
        plt.grid(True)

        # Save plot
        plt.savefig(f"plots/{target_sheet}_segmented_trend.png", bbox_inches='tight', dpi=300)
        plt.close()

        print(f"✅ Plot with segmented trendlines for '{target_sheet}' saved in 'plots/' folder.")

✅ Plot with segmented trendlines for 'ذهاب 2' saved in 'plots/' folder.


In [1]:
import pandas as pd
import jdatetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import os

# Load only the target sheet
excel_path = "pizometer_data.xlsx"
target_sheet = "ذهاب 2"
df = pd.read_excel(excel_path, sheet_name=target_sheet)

# Create folder for plots
os.makedirs("plots", exist_ok=True)

# Convert Shamsi (Jalali) date to Gregorian
def shamsi_to_gregorian_safe(date_str):
    try:
        year, month, day = map(int, str(date_str).split('/'))
        return jdatetime.date(year, month, day).togregorian()
    except:
        return None

# Validate required columns
required_columns = ['tarikh', 'taraz', 'subsidence']
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    print(f"⛔ Skipping: Missing columns {missing_cols} in sheet '{target_sheet}'")
else:
    # Process piezometer data
    df['tarikh_miladi'] = df['tarikh'].apply(shamsi_to_gregorian_safe)
    df = df.dropna(subset=['tarikh_miladi', 'taraz', 'subsidence']).copy()

    # Convert tarikh_miladi to pandas Timestamp
    df['tarikh_miladi'] = pd.to_datetime(df['tarikh_miladi'])
    df['taraz'] = pd.to_numeric(df['taraz'], errors='coerce')
    df['subsidence'] = pd.to_numeric(df['subsidence'], errors='coerce')
    df = df.dropna(subset=['taraz', 'subsidence'])

    if len(df) < 2:
        print(f"⚠️ Not enough data in '{target_sheet}' for plotting.")
    else:
        df = df.sort_values('tarikh_miladi')

        # Create figure and twin axes
        fig, ax1 = plt.subplots(figsize=(14, 4))
        ax2 = ax1.twinx()

        # Plot piezometer data (taraz) on left Y-axis
        ax1.scatter(df['tarikh_miladi'], df['taraz'], color='blue', label='Piezometer', alpha=1)

        # Plot subsidence data on right Y-axis
        ax2.scatter(df['date'], df['subsidence'], color='indianred', label='Subsidence', alpha=1)

        # Define periods for trendlines (same as original for piezometer)
        periods = [
            ('Piezometer Main (Excl. 2017 to 2020-05)', None, None),
            ('Piezometer Noise (2017 to 2020-05)', '2017-01-01', '2020-05-31')
        ]

        # Trendline colors
        trendline_colors = {
            'Piezometer Main (Excl. 2017 to 2020-05)': 'red',
            'Piezometer Noise (2017 to 2020-05)': 'green',
            'Subsidence': 'black'
        }

        # Process piezometer trendlines
        for label, start_date, end_date in periods:
            mask = pd.Series(True, index=df.index)
            if label == 'Piezometer Main (Excl. 2017 to 2020-05)':
                mask &= ~((df['tarikh_miladi'] >= pd.to_datetime('2017-01-01')) & 
                          (df['tarikh_miladi'] <= pd.to_datetime('2020-05-31')))
            else:
                if start_date:
                    mask &= df['tarikh_miladi'] >= pd.to_datetime(start_date)
                if end_date:
                    mask &= df['tarikh_miladi'] <= pd.to_datetime(end_date)
            
            df_period = df[mask].copy()
            
            if len(df_period) >= 2:
                x = mdates.date2num(df_period['tarikh_miladi'])
                y = df_period['taraz'].values

                start_year = df_period['tarikh_miladi'].min().year
                end_year = df_period['tarikh_miladi'].max().year

                if np.std(y) > 1e-6:
                    try:
                        coeffs = np.polyfit(x, y, 1)
                        slope_per_day = coeffs[0]
                        slope_per_year = slope_per_day * 365.25
                        trendline = np.poly1d(coeffs)
                        y_trend = trendline(x)
                        ax1.plot(df_period['tarikh_miladi'], y_trend, linestyle='--',
                                linewidth=2.5, color=trendline_colors[label],
                                label=f'{label} , rate: {slope_per_year:.2f} m/year)')
                       # label=f'{label} ({start_year}-{end_year}, rate: {slope_per_year:.2f} m/year)')
                    except np.linalg.LinAlgError:
                        print(f"⚠️ Piezometer trendline failed for '{label}' in '{target_sheet}' (SVD issue)")
                else:
                    print(f"⚠️ Not enough variation in '{label}' for piezometer trendline.")
            else:
                print(f"⚠️ Not enough data in '{label}' for piezometer trendline.")

        # Process subsidence trendline (full dataset, inspired by second code)
        x = mdates.date2num(df['date'])
        y = df['subsidence'].values
        if len(x) >= 2 and np.std(y) > 1e-6:
            try:
                coeffs = np.polyfit(x, y, 1)
                slope_per_day = coeffs[0]
                slope_per_year = slope_per_day * 365.25
                trendline = np.poly1d(coeffs)
                y_trend = trendline(x)
                ax2.plot(df['date'], y_trend, linestyle='--', linewidth=2.5,
                         color=trendline_colors['Subsidence'],
                         label=f'Subsidence Trend ({df["date"].min().year}-{df["date"].max().year}, rate: {slope_per_year:.2f} mm/year)')
            except np.linalg.LinAlgError:
                print(f"⚠️ Subsidence trendline failed for '{target_sheet}' (SVD issue)")
        else:
            print(f"⚠️ Not enough data or variation for subsidence trendline in '{target_sheet}'.")

        # Customize axes
       # ax1.set_xlabel('Gregorian Date')
        ax1.set_ylabel('Taraz (m)') #, color='blue')
        ax2.set_ylabel('Subsidence (mm)') #, color='indianred')
        ax1.tick_params(axis='y') #, labelcolor='blue')
        ax2.tick_params(axis='y') #, labelcolor='indianred')

        # Combine legends
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines1 + lines2, labels1 + labels2, loc='best')

        plt.title(f"Piezometer and Subsidence Observations ")
        plt.grid(True)

        # Save plot
        plt.savefig(f"plots/{target_sheet}_dual_axis_trend.png", bbox_inches='tight', dpi=300)
        plt.close()

        print(f"✅ Dual-axis plot with trendlines for '{target_sheet}' saved in 'plots/' folder.")

✅ Dual-axis plot with trendlines for 'ذهاب 2' saved in 'plots/' folder.


In [33]:
##### fine ####

In [10]:
import pandas as pd
import jdatetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import os

excel_path = "pizometer_data.xlsx"
target_sheet = "جمع آب"

# Create folder for combined plots
os.makedirs("combined_plots", exist_ok=True)

# Load the sheet
df = pd.read_excel(excel_path, sheet_name=target_sheet)

# Convert Shamsi (Jalali) date to Gregorian
def shamsi_to_gregorian_safe(date_str):
    try:
        year, month, day = map(int, str(date_str).split('/'))
        return jdatetime.date(year, month, day).togregorian()
    except:
        return None

# -------------------------------
# Process Piezometer Data
# -------------------------------
if 'tarikh' not in df.columns or 'taraz' not in df.columns:
    print(f"⛔ Missing 'tarikh' or 'taraz' in sheet '{target_sheet}'")
else:
    df['tarikh_miladi'] = df['tarikh'].apply(shamsi_to_gregorian_safe)
    df = df.dropna(subset=['tarikh_miladi', 'taraz']).copy()
    df['tarikh_miladi'] = pd.to_datetime(df['tarikh_miladi'])
    df['taraz'] = pd.to_numeric(df['taraz'], errors='coerce')
    df = df.dropna(subset=['taraz']).sort_values('tarikh_miladi')

    # Define periods for trendlines
    periods = [
        ('1965 to 1975', None, '1975-12-31'),
        ('1982 to 1997', '1982-01-01', '1997-12-31'),
        ('2003 to 2022', '2003-01-01', None)
    ]
    trendline_colors = {
        '1965 to 1975': 'red',
        '1982 to 1997': 'green',
        '2003 to 2022': 'purple'
    }

    # -------------------------------
    # Process Subsidence Data
    # -------------------------------
    if 'date' not in df.columns or 'subsidence' not in df.columns:
        print(f"⚠️ Missing 'date' or 'subsidence' for subsidence in '{target_sheet}'")
        df_subs = None
    else:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df['subsidence'] = pd.to_numeric(df['subsidence'], errors='coerce')
        df_subs = df.dropna(subset=['date', 'subsidence']).sort_values('date')

    # -------------------------------
    # Plot Combined Figure
    # -------------------------------
    fig, ax1 = plt.subplots(figsize=(14, 4))

    # --- Piezometer left Y-axis ---
    ax1.scatter(df['tarikh_miladi'], df['taraz'], color='blue', label='Piezometer')
   # ax1.set_xlabel('Gregorian Date')
    ax1.set_ylabel('Taraz (m)') #, color='blue')
    ax1.tick_params(axis='y') #, labelcolor='blue')

    # Trendlines for piezometer
    for label, start_date, end_date in periods:
        mask = pd.Series(True, index=df.index)
        if start_date:
            mask &= df['tarikh_miladi'] >= pd.to_datetime(start_date)
        if end_date:
            mask &= df['tarikh_miladi'] <= pd.to_datetime(end_date)
        df_period = df[mask].copy()
        if len(df_period) >= 2 and np.std(df_period['taraz'].values) > 1e-6:
            x = mdates.date2num(df_period['tarikh_miladi'])
            y = df_period['taraz'].values
            coeffs = np.polyfit(x, y, 1)
            slope_per_year = coeffs[0]*365.25
            trendline = np.poly1d(coeffs)
            ax1.plot(df_period['tarikh_miladi'], trendline(x), '--', color=trendline_colors[label],
                     label=f'{label} Trend ({slope_per_year:.2f} m/yr)')

    # --- Subsidence right Y-axis ---
    if df_subs is not None and len(df_subs) >= 2:
        ax2 = ax1.twinx()
        ax2.scatter(df_subs['date'], df_subs['subsidence'], color='indianred', label='Subsidence')
        x_subs = mdates.date2num(df_subs['date'])
        y_subs = df_subs['subsidence'].values
        coeffs_subs = np.polyfit(x_subs, y_subs, 1)
        slope_per_year_subs = coeffs_subs[0]*365.25
        trendline_subs = np.poly1d(coeffs_subs)
        ax2.plot(df_subs['date'], trendline_subs(x_subs), '--', color='black',
                 label=f'Subsidence Trend ({slope_per_year_subs:.2f} mm/yr)')
        ax2.set_ylabel('Subsidence (mm)') #, color='indianred')
        ax2.tick_params(axis='y') #, labelcolor='indianred')

    # --- Legends ---
    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels() if df_subs is not None else ([], [])
    ax1.legend(handles1 + handles2, labels1 + labels2, loc='upper left')

    plt.title(f"Piezometer and Subsidence")
    plt.grid(True)
    plt.tight_layout()

    # Save figure
    plt.savefig(f"combined_plots/{target_sheet}_combined.png", bbox_inches='tight', dpi=300)
    plt.close()

    print(f"✅ Combined plot saved in 'combined_plots/' folder.")


✅ Combined plot saved in 'combined_plots/' folder.


In [44]:
import pandas as pd
import jdatetime
import matplotlib.pyplot as plt
import os
import matplotlib.dates as mdates
import numpy as np

# Load the Excel file
excel_path = "pizometer_data.xlsx"  # Update with your actual file name
sheets = pd.read_excel(excel_path, sheet_name=None)

output_dir = "subsidence_plots"
os.makedirs(output_dir, exist_ok=True)

# Function to strip whitespace from column names
def clean_columns(df):
    df.columns = df.columns.str.strip()  # Strip leading/trailing whitespace
    return df

# Process each sheet
for sheet_name, df in sheets.items():
    print(f"📄 Processing sheet: {sheet_name}")
    
    # Clean column names
    df = clean_columns(df)

    print(f"Columns: {df.columns}")

    if 'date' not in df.columns or 'subsidence' not in df.columns:
        print(f"⚠️ Skipped '{sheet_name}': Missing required columns.")
        continue

    # Convert date to datetime and check
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    print(f"First few date values after conversion: {df['date'].head()}")

    df = df.dropna(subset=['date', 'subsidence'])

    if df.empty:
        print(f"⚠️ Skipped '{sheet_name}': No valid data after cleaning.")
        continue

    print(f"First few rows of cleaned data:\n{df.head()}")

    df = df.sort_values('date')
    x = mdates.date2num(df['date'])
    y = df['subsidence'].astype(float).values

    # Calculate trendline and annual rate
    if len(x) >= 2:
        coeffs = np.polyfit(x, y, 1)
        trendline = np.poly1d(coeffs)
        y_trend = trendline(x)
        # Slope is in mm/day, convert to mm/year
        annual_rate = coeffs[0] * 365.25  # Use 365.25 days/year for accuracy
        annual_rate_str = f"{annual_rate:.2f} mm/year"

    plt.figure(figsize=(14, 4))
    plt.scatter(df['date'], y, color='indianred', label='Subsidence Observations')
    if len(x) >= 2:
        plt.plot(df['date'], y_trend, color='black', linestyle='--', 
                label=f'Trendline (Rate: {annual_rate_str})')

    plt.xlabel('Gregorian Date')
    plt.ylabel('Subsidence (mm)')
    plt.title("InSAR Observation")
    plt.legend()
    # Enhanced grid with visible style
    plt.grid(True, which='both', linestyle='-', linewidth=0.8, color='black', alpha=0.7)

    # Save the plot
    safe_name = sheet_name.strip()  # Ensure sheet name is clean and safe
    plt.savefig(f"{output_dir}/{safe_name}.png", bbox_inches='tight', dpi=300)
    plt.close()

print("✅ Subsidence plots saved in 'subsidence_plots/' folder.")

📄 Processing sheet: بی نام
Columns: Index(['سال', 'ماه', 'روز', 'سال_آ', 'کد_مح', 'محدود', 'نام_م', 'سطح_آ',
       'XUTM', 'YUTM', 'علت_ع', 'تراز__', 'ملاحظ', 'تيسن__', 'تيسن_1', 'taraz',
       'tarikh', 'date', 'subsidence'],
      dtype='object')
First few date values after conversion: 0   2015-02-20
1   2015-03-04
2   2015-03-28
3   2015-04-21
4   2015-05-15
Name: date, dtype: datetime64[ns]
First few rows of cleaned data:
    سال  ماه  روز    سال_آ  کد_مح          محدود نام_م  سطح_آ    XUTM  \
0  1375    7   29  1375-76   6007  مشهد - چناران        35.69  684800   
1  1375    8   29  1375-76   6007  مشهد - چناران        36.31  684800   
2  1375    9   20  1375-76   6007  مشهد - چناران        36.36  684800   
3  1375   10    3  1375-76   6007  مشهد - چناران        36.55  684800   
4  1375   12   20  1375-76   6007  مشهد - چناران        37.33  684800   

      YUTM  علت_ع  تراز__  ملاحظ  تيسن__  تيسن_1  taraz      tarikh  \
0  4054000      0       0      0       0       0 -35.69   

In [45]:

import pandas as pd
import jdatetime
import matplotlib.pyplot as plt
import os
import matplotlib.dates as mdates
import numpy as np

# Load the Excel file
excel_path = "pizometer_data.xlsx"  # Update with your actual file name
sheets = pd.read_excel(excel_path, sheet_name=None)

output_dir = "subsidence_plots"
os.makedirs(output_dir, exist_ok=True)

# Function to strip whitespace from column names
def clean_columns(df):
    df.columns = df.columns.str.strip()  # Strip leading/trailing whitespace
    return df

# Process each sheet
for sheet_name, df in sheets.items():
    print(f"📄 Processing sheet: {sheet_name}")
    
    # Clean column names
    df = clean_columns(df)

    print(f"Columns: {df.columns}")

    if 'date' not in df.columns or 'subsidence' not in df.columns:
        print(f"⚠️ Skipped '{sheet_name}': Missing required columns.")
        continue

    # Convert date to datetime and check
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    print(f"First few date values after conversion: {df['date'].head()}")

    df = df.dropna(subset=['date', 'subsidence'])

    if df.empty:
        print(f"⚠️ Skipped '{sheet_name}': No valid data after cleaning.")
        continue

    print(f"First few rows of cleaned data:\n{df.head()}")

    df = df.sort_values('date')
    x = mdates.date2num(df['date'])
    y = df['subsidence'].astype(float).values

    if len(x) >= 2:
        coeffs = np.polyfit(x, y, 1)
        trendline = np.poly1d(coeffs)
        y_trend = trendline(x)

    plt.figure(figsize=(14, 4))
    plt.scatter(df['date'], y, color='indianred', label='Subsidence Observations')
    if len(x) >= 2:
        plt.plot(df['date'], y_trend, color='black', linestyle='--', label='Trendline')

    plt.xlabel('Gregorian Date')
    plt.ylabel('Subsidence (mm)')
    plt.title("InSAR observation")
    plt.legend()
    plt.grid(True)

    # Save the plot
    safe_name = sheet_name.strip()  # Ensure sheet name is clean and safe
    plt.savefig(f"{output_dir}/{safe_name}.png", bbox_inches='tight', dpi=300)
    plt.close()

print("✅ Subsidence plots saved in 'subsidence_plots/' folder.")

📄 Processing sheet: بی نام
Columns: Index(['سال', 'ماه', 'روز', 'سال_آ', 'کد_مح', 'محدود', 'نام_م', 'سطح_آ',
       'XUTM', 'YUTM', 'علت_ع', 'تراز__', 'ملاحظ', 'تيسن__', 'تيسن_1', 'taraz',
       'tarikh', 'date', 'subsidence'],
      dtype='object')
First few date values after conversion: 0   2015-02-20
1   2015-03-04
2   2015-03-28
3   2015-04-21
4   2015-05-15
Name: date, dtype: datetime64[ns]
First few rows of cleaned data:
    سال  ماه  روز    سال_آ  کد_مح          محدود نام_م  سطح_آ    XUTM  \
0  1375    7   29  1375-76   6007  مشهد - چناران        35.69  684800   
1  1375    8   29  1375-76   6007  مشهد - چناران        36.31  684800   
2  1375    9   20  1375-76   6007  مشهد - چناران        36.36  684800   
3  1375   10    3  1375-76   6007  مشهد - چناران        36.55  684800   
4  1375   12   20  1375-76   6007  مشهد - چناران        37.33  684800   

      YUTM  علت_ع  تراز__  ملاحظ  تيسن__  تيسن_1  taraz      tarikh  \
0  4054000      0       0      0       0       0 -35.69   

In [18]:
import pandas as pd
import jdatetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import os
  

# ------------------------------
# Load only the target sheet
# ------------------------------
excel_path = "pizometer_data.xlsx"
target_sheet = "ذهاب 2"
df = pd.read_excel(excel_path, sheet_name=target_sheet)

# Create folder for plots
os.makedirs("plots", exist_ok=True)

# ------------------------------
# Convert Shamsi (Jalali) to Gregorian
# ------------------------------
def shamsi_to_gregorian_safe(date_str):
    try:
        year, month, day = map(int, str(date_str).split('/'))
        return jdatetime.date(year, month, day).togregorian()
    except:
        return None

# ------------------------------
# Validate required columns
# ------------------------------
required_columns = ['tarikh', 'taraz', 'subsidence', 'date']
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    print(f"⛔ Skipping: Missing columns {missing_cols} in sheet '{target_sheet}'")
else:
    # ------------------------------
    # Prepare DataFrames separately
    # ------------------------------
# ------------------------------

    # Piezometer
    piezo = df[['tarikh', 'taraz']].copy()
    piezo['tarikh_miladi'] = piezo['tarikh'].apply(shamsi_to_gregorian_safe)
    piezo['tarikh_miladi'] = pd.to_datetime(piezo['tarikh_miladi'], errors='coerce')
    piezo['taraz'] = pd.to_numeric(piezo['taraz'], errors='coerce')
    piezo = piezo.dropna(subset=['tarikh_miladi', 'taraz']).sort_values('tarikh_miladi')

    # Subsidence
    subs = df[['date', 'subsidence']].copy()
    subs['date'] = pd.to_datetime(subs['date'], errors='coerce')
    subs['subsidence'] = pd.to_numeric(subs['subsidence'], errors='coerce')
    subs = subs.dropna(subset=['date', 'subsidence']).sort_values('date')

    # ------------------------------
    # Plot setup
    # ------------------------------
    fig, ax1 = plt.subplots(figsize=(14, 4))
    ax2 = ax1.twinx()

    # Scatter plots
    if not piezo.empty:
        ax1.scatter(piezo['tarikh_miladi'], piezo['taraz'], color='blue', label='Piezometer', alpha=1)
    if not subs.empty:
        ax2.scatter(subs['date'], subs['subsidence'], color='indianred', label='Subsidence', alpha=1)

    # ------------------------------
    # Piezometer Trendlines
    # ------------------------------
    periods = [
        ('Piezometer Main (Excl. 2017 to 2020-05)', None, None),
        ('Piezometer Noise (2017 to 2020-05)', '2017-01-01', '2020-05-31')
    ]

    trendline_colors = {
        'Piezometer Main (Excl. 2017 to 2020-05)': 'red',
        'Piezometer Noise (2017 to 2020-05)': 'green',
        'Subsidence': 'black'
    }

    for label, start_date, end_date in periods:
        if piezo.empty:
            continue

        mask = pd.Series(True, index=piezo.index)

        if label == 'Piezometer Main (Excl. 2017 to 2020-05)':
            mask &= ~((piezo['tarikh_miladi'] >= pd.to_datetime('2017-01-01')) &
                      (piezo['tarikh_miladi'] <= pd.to_datetime('2020-05-31')))
        else:
            if start_date:
                mask &= piezo['tarikh_miladi'] >= pd.to_datetime(start_date)
            if end_date:
                mask &= piezo['tarikh_miladi'] <= pd.to_datetime(end_date)

        df_period = piezo[mask].copy()

        if len(df_period) >= 2:
            x = mdates.date2num(df_period['tarikh_miladi'])
            y = df_period['taraz'].values

            if np.std(y) > 1e-6:
                try:
                    coeffs = np.polyfit(x, y, 1)
                    slope_per_day = coeffs[0]
                    slope_per_year = slope_per_day * 365.25
                    trendline = np.poly1d(coeffs)
                    y_trend = trendline(x)
                    ax1.plot(df_period['tarikh_miladi'], y_trend, linestyle='--',
                             linewidth=2.5, color=trendline_colors[label],
                             label=f'{label}, rate: {slope_per_year:.2f} m/year')
                except np.linalg.LinAlgError:
                    print(f"⚠️ Piezometer trendline failed for '{label}' in '{target_sheet}' (SVD issue)")
            else:
                print(f"⚠️ Not enough variation in '{label}' for piezometer trendline.")

    # ------------------------------
    # Subsidence Trendline (Full)
    # ------------------------------
    if len(subs) >= 2:
        x = mdates.date2num(subs['date'])
        y = subs['subsidence'].values

        if np.std(y) > 1e-6:
            try:
                coeffs = np.polyfit(x, y, 1)
                slope_per_day = coeffs[0]
                slope_per_year = slope_per_day * 365.25
                trendline = np.poly1d(coeffs)
                y_trend = trendline(x)
                ax2.plot(subs['date'], y_trend, linestyle='--', linewidth=2.5,
                         color=trendline_colors['Subsidence'],
                         label=f'Subsidence Trend ({subs["date"].min().year}-{subs["date"].max().year}, rate: {slope_per_year:.2f} mm/year)')
            except np.linalg.LinAlgError:
                print(f"⚠️ Subsidence trendline failed for '{target_sheet}' (SVD issue)")
        else:
            print(f"⚠️ Not enough variation for subsidence trendline in '{target_sheet}'.")

    # ------------------------------
    # Final Styling
    # ------------------------------
    ax1.set_ylabel('Taraz (m)')
    ax2.set_ylabel('Subsidence (mm)')
    ax1.tick_params(axis='y')
    ax2.tick_params(axis='y')

    # Combine legends
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='best')

    plt.title(f"Piezometer and Subsidence Observations")
    plt.grid(True)
# Force x-axis to start from 2015
    max_date = max(
        piezo['tarikh_miladi'].max() if not piezo.empty else pd.to_datetime("2015-01-01"),
        subs['date'].max() if not subs.empty else pd.to_datetime("2015-01-01")
    )
    ax1.set_xlim(left=pd.to_datetime("2015-01-01"), right=max_date)

    # Save
    plt.savefig(f"plots/{target_sheet}_dual_axis_trend.png", bbox_inches='tight', dpi=300)
    plt.close()

    print(f"✅ Dual-axis plot with trendlines for '{target_sheet}' saved in 'plots/' folder.")


✅ Dual-axis plot with trendlines for 'ذهاب 2' saved in 'plots/' folder.


In [7]:
df

,سال,ماه,روز,سال_آ,کد_مح,محدود,نام_م,سطح_آ,XUTM,YUTM,علت_ع,تراز__,ملاحظ,تيسن__,تيسن_1,taraz,tarikh,date,subsidence
0,1389.0,9.0,27.0,1389-90,6007.0,مشهد - چناران,ذهاب 2,13.49,696597.0,4056590.0,0.0,1148.64,0.0,0.0,0.0,1135.15,1389/9/27,2015-02-20,0.00
1,1389.0,11.0,11.0,1389-90,6007.0,مشهد - چناران,ذهاب 2,17.79,696597.0,4056590.0,0.0,1148.64,0.0,0.0,0.0,1130.85,1389/11/11,2015-03-04,-3.31
2,1389.0,8.0,8.0,1389-90,6007.0,مشهد - چناران,ذهاب 2,11.92,696597.0,4056590.0,0.0,1148.64,0.0,0.0,0.0,1136.72,1389/8/8,2015-03-28,-4.43
3,1389.0,10.0,13.0,1389-90,6007.0,مشهد - چناران,ذهاب 2,16.33,696597.0,4056590.0,0.0,1148.64,0.0,0.0,0.0,1132.31,1389/10/13,2015-04-21,-16.00
4,1389.0,7.0,14.0,1389-90,6007.0,مشهد - چناران,ذهاب 2,14.79,696597.0,4056590.0,0.0,1148.64,0.0,0.0,0.0,1133.85,1389/7/14,2015-05-15,-24.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-23,-659.53
227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-04,-660.71
228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-16,-664.04
229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-09,-668.50


In [19]:
import pandas as pd
import jdatetime
import matplotlib.pyplot as plt
import os
import matplotlib.dates as mdates
import numpy as np

# Load the Excel file
excel_path = "pizometer_data.xlsx"  # Update with your actual file name
sheets = pd.read_excel(excel_path, sheet_name=None)

output_dir = "subsidence_plots"
os.makedirs(output_dir, exist_ok=True)

# Function to strip whitespace from column names
def clean_columns(df):
    df.columns = df.columns.str.strip()  # Strip leading/trailing whitespace
    return df

# Process each sheet
for sheet_name, df in sheets.items():
    print(f"📄 Processing sheet: {sheet_name}")
    
    # Clean column names
    df = clean_columns(df)

    print(f"Columns: {df.columns}")

    if 'date' not in df.columns or 'subsidence' not in df.columns:
        print(f"⚠️ Skipped '{sheet_name}': Missing required columns.")
        continue

    # Convert date to datetime and check
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    print(f"First few date values after conversion: {df['date'].head()}")

    df = df.dropna(subset=['date', 'subsidence'])

    if df.empty:
        print(f"⚠️ Skipped '{sheet_name}': No valid data after cleaning.")
        continue

    print(f"First few rows of cleaned data:\n{df.head()}")

    df = df.sort_values('date')
    x = mdates.date2num(df['date'])
    y = df['subsidence'].astype(float).values

    # Calculate trendline and annual rate
    if len(x) >= 2:
        coeffs = np.polyfit(x, y, 1)
        trendline = np.poly1d(coeffs)
        y_trend = trendline(x)
        # Slope is in mm/day, convert to mm/year
        annual_rate = coeffs[0] * 365.25  # Use 365.25 days/year for accuracy
        annual_rate_str = f"{annual_rate:.0f} mm/year"

    plt.figure(figsize=(14, 4))
    plt.scatter(df['date'], y, color='indianred', label='Subsidence Observations')
    if len(x) >= 2:
        plt.plot(
            df['date'], y_trend, color='black', linestyle='--',
            label=f'Trendline (Rate: {annual_rate_str})'
        )

    plt.ylabel('Subsidence (mm)')
    plt.title("InSAR Observation")
    plt.legend()
    plt.grid(True)
# Force x-axis to start from 2015
    max_date = max(
        piezo['tarikh_miladi'].max() if not piezo.empty else pd.to_datetime("2015-01-01"),
        subs['date'].max() if not subs.empty else pd.to_datetime("2015-01-01")
    )
    ax1.set_xlim(left=pd.to_datetime("2015-01-01"), right=max_date)

    # Save the plot
    safe_name = sheet_name.strip()  # Ensure sheet name is clean and safe
    plt.savefig(f"{output_dir}/{safe_name}.png", bbox_inches='tight', dpi=300)
    plt.close()

print("✅ Subsidence plots saved in 'subsidence_plots/' folder.")


📄 Processing sheet: بی نام
Columns: Index(['سال', 'ماه', 'روز', 'سال_آ', 'کد_مح', 'محدود', 'نام_م', 'سطح_آ',
       'XUTM', 'YUTM', 'علت_ع', 'تراز__', 'ملاحظ', 'تيسن__', 'تيسن_1', 'taraz',
       'tarikh', 'date', 'subsidence'],
      dtype='object')
First few date values after conversion: 0   2015-02-20
1   2015-03-04
2   2015-03-28
3   2015-04-21
4   2015-05-15
Name: date, dtype: datetime64[ns]
First few rows of cleaned data:
    سال  ماه  روز    سال_آ  کد_مح          محدود نام_م  سطح_آ    XUTM  \
0  1375    7   29  1375-76   6007  مشهد - چناران        35.69  684800   
1  1375    8   29  1375-76   6007  مشهد - چناران        36.31  684800   
2  1375    9   20  1375-76   6007  مشهد - چناران        36.36  684800   
3  1375   10    3  1375-76   6007  مشهد - چناران        36.55  684800   
4  1375   12   20  1375-76   6007  مشهد - چناران        37.33  684800   

      YUTM  علت_ع  تراز__  ملاحظ  تيسن__  تيسن_1  taraz      tarikh  \
0  4054000      0       0      0       0       0 -35.69   